# Using the Model for Prediction and Streamlit Implementation

The model was trained and tested on data from **April 2017 to March 2025**. It accounts for **seasonal fluctuations** as well as complex input data such as **binary water masks with limited structural information**.

To make a prediction, the method `model.predict()` must be used. It is important to ensure that the input data (e. g. preprocessed satellite images and exogenous factors) **match the structure and dimensions** used during training.

### Requirements for Predictions

- **Satellite Images**  
  At least the **last 5 images** are required.

- **Exogenous Factors**  
  Relevant data such as **precipitation** and **discharge**, which can be obtained from:
  - Local authorities (e. g. *Gewässerkundlicher Dienst Bayern*)
  - Web scraping sources (e. g. **OpenMeteo**). Note: Data sources may not always be consistent or fully aligned.

### Implementation in Streamlit

To integrate the model into **Streamlit**, a `.py` script is required that includes the described workflow.

A **customizable template** will be provided to support this implementation.


In [ ]:
#Import packages

import ee
import geemap
import os

import pandas as pd
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt

import rasterio
import glob

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import folium
from folium.raster_layers import ImageOverlay


In [ ]:
#Step 1: Get Satellite Imagery from Google Earth Engine


# Initialize Earth Engine API
# Make sure to authenticate if you haven't done so already
#ee.Authenticate()  # Uncomment this line if you need to authenticate
# Initialize the Earth Engine library

ee.Initialize()


#Define the coordinates of the bounding box (xmin, ymin) and (xmax, ymax) of your project area
#Coordinates are in EPSG:4326
#Example coordinates for a bounding box around Vorderriß, Germany
bbox_coords = ((11.437948183612978, 47.55891195678332), (11.484581611702358, 47.56320074444652))
geometry = ee.Geometry.Polygon([
    [
        [bbox_coords[0][0], bbox_coords[0][1]],  # (xmin, ymin)
        [bbox_coords[0][0], bbox_coords[1][1]],  # (xmin, ymax)
        [bbox_coords[1][0], bbox_coords[1][1]],  # (xmax, ymax)
        [bbox_coords[1][0], bbox_coords[0][1]],  # (xmax, ymin)
        [bbox_coords[0][0], bbox_coords[0][1]],  # Closing the polygon
    ]
])

# Define a function to calculate snow fraction of each image. Snow has similiar reflectance properties as water.
def add_snow_fraction(image):
    scl = image.select("SCL")
    snow_mask = scl.eq(11)  # SCL = 11 means snow/ice

    snow_fraction = snow_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=20,
        maxPixels=1e8
    ).get('SCL')  # result is a number between 0 and 1

    return image.set('snow_fraction', snow_fraction)


# Define NDWI calculation function (normalized difference using B3 and B8)
def calculateNDWI(image):
    ndwi = image.normalizedDifference(["B3", "B8"]).rename("NDWI")
    return image.addBands(ndwi)

# Define a function to clip the image to your geometry
def clip_image(image):
    return image.clip(geometry)

# Create a function to assign a chronological image_id based on the collection index
def create_feature_with_index(image, index):
    # Convert the index to a string to be used as the image ID
    image_id = ee.Number(index)  # Using index as the image ID
    timestamp = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')  # Format timestamp as YYYY-MM-DD
    
    # Create a feature with the image ID and timestamp
    feature = ee.Feature(None, {
        'image_id': image_id,
        'timestamp': timestamp
    })
    return feature

# Function to assign a chronological index to each image in the collection
def add_index_to_collection(image_collection):
    # Create a list of features with image_id as index
    def assign_index(image, index):
        return create_feature_with_index(image, index)
    
    # Map the function over the collection and add indices
    feature_collection = image_collection.map(lambda image: assign_index(image, image_collection.toList(image_collection.size()).indexOf(image)))
    return feature_collection


# Step 1. 1: Load Sentinel-2 image collection. Filter by date, bounds, and cloud cover.
sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate('2015-06-27', '2025-03-31') \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  

# Add snow fraction to each image
sentinel_snow_filtered = sentinel.map(add_snow_fraction)

# Filter images with < 3 % snow
sentinel_clean = sentinel_snow_filtered.filter(ee.Filter.lt('snow_fraction', 0.03))

processed_images = sentinel_clean.map(clip_image).map(calculateNDWI)


# Map the function over the processed image collection to create a FeatureCollection with chronological image_ids
features = add_index_to_collection(processed_images)

# Step 1. 2: Export images and metadata to Google Drive. If you authenticated with your Google account in Google Earth Engine, the images will be saved in your Google Drive.

# Export the FeatureCollection as a CSV to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=features,
    description='Image_Metadata',
    fileFormat='CSV',
    folder='Satellite_metadata',  # Folder in your Google Drive
    fileNamePrefix='satellite_metadata_all',
)

# Start the export task
export_task.start()

print("Export task for image metadata started. Monitor Earth Engine for task completion.")


# Export each image in the collection
def export_image(image, idx):  
    # Export the image to a GeoTIFF using Earth Engine's export method
    export_task = ee.batch.Export.image.toDrive(
        image=image.select("NDWI"),
        description=None,
        folder="GEE_Images_all",
        fileNamePrefix=f"NDWI_{idx}",
        region=geometry,
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e8  # Increase the max pixels if necessary
    )
    export_task.start()  # Start the export task
    print(f"Exporting {idx}")

# Convert the image collection to a list of images. This triggers the actual fetching of the collection.
image_list = processed_images.toList(processed_images.size())

# Loop through the image collection and export each image
for idx in range(image_list.size().getInfo()):
    ee_image = ee.Image(image_list.get(idx))  # Get the image by its index in the list
    export_image(ee_image, idx)

print("Export tasks started. Monitor Earth Engine for task completion.")


In [ ]:
# Step 1. 3: Add an index column to the CSV file exported from Earth Engine

metadata_path = ''  
metadata_df = pd.read_csv(metadata_path)

metadata_df['Index'] = range(len(metadata_df)) #Creating an index column with ascending values

output_path = ''  # Save the modified DataFrame to a new CSV file
metadata_df.to_csv(output_path, index=False)

print(f"Added new column 'Index' and saved it in: {output_path}")


In [ ]:
# Define a function to extract the ID from the filename
# This function assumes the filename format is "NDWI_<ID>.tif"
def extract_id_from_filename(filename):
    return int(filename.split('_')[1].split('.')[0]) 

# Define a function to get the timestamp from the image ID and double check if the ID is in the new CSV file 
def get_timestamp_from_id(image_id):
    timestamp_data = pd.read_csv('')  
    row = timestamp_data[timestamp_data['Index'] == image_id]
    if not row.empty:
        return row['timestamp'].values[0]
    else:
        return None  # If the ID is not found, return None

# Define a function to process NDWI images and create water masks
def process_images_and_create_water_masks(folder_path):
    output_dir_water_masks = '/'  # folder for saving water masks

    # Make sure that the folder exists
    if not os.path.exists(output_dir_water_masks):
        os.makedirs(output_dir_water_masks)

    # Order by numeric ID
    image_files =[f for f in os.listdir(folder_path) if f.lower().endswith(('.tif', '.tiff'))]
    image_files.sort(key=extract_id_from_filename)  

    # Loop through each image file
    for filename in image_files:
        if filename.lower().endswith(('.tif', '.tiff')):
            image_id = extract_id_from_filename(filename)
            timestamp = get_timestamp_from_id(image_id)
            if timestamp is None:
                print(f"No timestamp for {filename}. Pass.")
                continue
            
            # Open the NDWI image using GDAL
            file_path = os.path.join(folder_path, filename)
            dataset = gdal.Open(file_path)
            if dataset is None:
                print(f"There was a mistake while opening: {filename}.")
                continue
            
            # Read the NDWI band and calculate the individual threshold. 
            # For the project area possible fraction of water in the image is between 5,8 % and 6,5 %. 
            # Assuming a unique threshold for each image is not recommended.
            band = dataset.GetRasterBand(1)
            ndwi = band.ReadAsArray()
            
            threshold = np.percentile(ndwi, 93)
            if np.isnan(threshold):
                threshold = 0.05
                print(f"Threshold for {filename} is NaN. Set to {threshold}.")
            # Check if the threshold is within the expected range. Unless change it.
            elif threshold < -0.1 or threshold > 0.1:
                # Get the sign of the threshold (-1, +1)
                sign = np.sign(threshold) 
                
                # If the threshold is negative, set it to -0.1, but proportional to the size of the threshold
                if threshold < -0.1:
                    threshold = -0.1 + (threshold + 0.1) * 0.7
                # If the threshold is positive, set it to 0.1, but proportional to the size of the threshold
                elif threshold > 0.1:
                    threshold = 0.1 - (threshold - 0.1) * 0.7

                threshold = np.clip(threshold, -0.1, 0.1)
                
                print(f"Invalid threshold for {filename}: Set to {threshold}.")
            else:
                print(f"Valid threshold for {filename} is {threshold}.")

            # Create a binary water mask based on the threshold and save it as a new GeoTIFF 
            water_mask = ndwi > threshold
        
            water_mask_filename = os.path.join(output_dir_water_masks, f"water_mask_{os.path.splitext(filename)[0]}.tif")
            driver = gdal.GetDriverByName('GTiff')
            mask_dataset = driver.Create(
                water_mask_filename,
                dataset.RasterXSize,
                dataset.RasterYSize,
                1,
                gdal.GDT_Byte
            )
            mask_dataset.SetGeoTransform(dataset.GetGeoTransform())
            mask_dataset.SetProjection(dataset.GetProjection())
            mask_band = mask_dataset.GetRasterBand(1)
            mask_band.WriteArray(water_mask.astype(np.uint8)) 
            mask_dataset = None
            print(f"Saved water mask: {water_mask_filename}")

    print("All water masks created and saved.")
    return


folder_path = ''  # Path to the folder with NDWI images

# Call the function to process images and create water masks
process_images_and_create_water_masks(folder_path)


In [ ]:
# Step 3. 1: Load water masks from the folder and add a new dimension to the images because the model requires 3D input or 4D input

def extract_id_from_filename(filename):
    return int(filename.split('_')[-1].split('.')[0]) 

# Define a function to load all TIF images from a directory in sorted order
def load_all_tif_images(directory):
    image_files = [f for f in os.listdir(directory) if f.lower().endswith('.tif')]
    image_files.sort(key=extract_id_from_filename)
    
    images = []
    for filename in image_files:
        file_path = os.path.join(directory, filename)
        with rasterio.open(file_path) as src:
            image = src.read(1)  
            image = np.expand_dims(image, axis=-1) # Add a new dimension
            images.append(image)
    return np.array(images)

images_directory = "/" # Path to the folder with water masks
images = load_all_tif_images(images_directory)

print(f"Amount of loaded images: {len(images)}")

In [ ]:
#Step 3. 2: Prepare exogenous factors for data set

exo_factors_df = pd.read_csv("")  # Path to the CSV file with exogenous factors

# Convert the 'timestamp' column to datetime format
exo_factors_df['timestamp'] = pd.to_datetime(exo_factors_df['timestamp'])

# Calculate the time difference in days between consecutive timestamps
exo_factors_df['time_diff_days'] = exo_factors_df['timestamp'].diff().dt.days

# Extract the month from the timestamp and one-hot encode it and save the columns names
exo_factors_df['month'] = exo_factors_df['timestamp'].dt.month
month_one_hot = pd.get_dummies(exo_factors_df['month'], prefix='month')
month_one_hot_columns = list(month_one_hot.columns)

# Add the one-hot encoded month columns to the DataFrame
exo_factors_df = pd.concat([exo_factors_df, month_one_hot], axis=1)

# Drop the original month column
exo_factors_df.drop(columns=['month'], inplace=True)

# Convert other important columns of the DataFrame. Save only the columns that are needed for the model.
exo_factors_df['discharge'] = pd.to_numeric(exo_factors_df['delta_discharge'], errors='coerce')
exo_factors_df['precipitation'] = pd.to_numeric(exo_factors_df['delta_precipitation'], errors='coerce')
exo_factors_df['prec_extreme'] = pd.to_numeric(exo_factors_df['extreme_precipitation'], errors='coerce')
exo_factors_df['disc_extreme'] = pd.to_numeric(exo_factors_df['extreme_discharge'], errors='coerce')

exo_factors_df = exo_factors_df[['timestamp', 'image_id', 'discharge', 'precipitation', 'prec_extreme', 'disc_extreme', 'time_diff_days'] + list(month_one_hot.columns)]
exo_factors_df.fillna(0, inplace=True)
print(exo_factors_df.head())

# Convert the DataFrame to a NumPy array because the model requires 3D input or 4D input
exo_factors = exo_factors_df[['discharge', 'precipitation', 'prec_extreme', 'disc_extreme', 'time_diff_days'] + list(month_one_hot_columns)].values
print(exo_factors.shape)


In [ ]:
# Load model and check summary especially the shape of the input layer
model = load_model('')
model.summary()

Future exogenous values required for forecasting can be manually added—either by editing the CSV file or appending to the input array—or automatically calculated based on user inputs provided through the Streamlit interface.

In [ ]:
def create_test_sequences_with_future_exo(images, exo_factors, time_steps=5):
    """
    Create sequences of images and exogenous factors for testing. The shape of the input data must be the same as in the training data.
    Args:
        images (np.array): Array of images.
        exo_factors_test (np.array): Array of exogenous factors.
        time_steps (int): Number of time steps for the sequence.
        return_y (bool): Whether to return the target variable Y.
    Returns:
        tuple: Tuple containing the sequences of images, exogenous factors, future exogenous factors, and target variable Y (if return_y is True).  
    """
    X_image_seqs = []
    X_exo_seqs = []
    future_exo_seqs = []

    max_start = len(images) - time_steps - 1  

    for i in range(max_start):
        img_seq = images[i:i + time_steps]  
        exo_seq = exo_factors[i:i + time_steps]  
        future_exo = exo_factors[i + time_steps]  
        
        X_image_seqs.append(img_seq)
        X_exo_seqs.append(exo_seq)
        future_exo_seqs.append(future_exo)

        return (
            np.array(X_image_seqs), 
            np.array(X_exo_seqs), 
            np.array(future_exo_seqs)
        )

In [ ]:
# Create sequences with future exogenous factors
X_img_seq, X_exo_seq, future_exo_seq = create_test_sequences_with_future_exo(images, exo_factors, time_steps=5)

# Convert the sequences to float32
X_img_seq = X_img_seq.astype('float32')
X_exo_seq = X_exo_seq.astype('float32')
future_exo_seq = future_exo_seq.astype('float32')

# Predict the image(s) using the model
predicted_images = model.predict((X_img_seq, X_exo_seq, future_exo_seq))

In [ ]:
# Create a Heatmap of the river shift by comparing the predicted image with the last actual image
# Depending on the amount of satellite images you were providing to the model, you can change the index of the last actual image
# index of the last predicted image is: len(images) - time_steps - 1

last_actual_image = images[i-time_steps-1].squeeze()
predicted_image = predicted_images[i].squeeze()

# Calculate the difference to visualize the river shift and create a mask to highlight significant changes
river_shift = predicted_image - last_actual_image
alpha_mask = np.where((river_shift < -0.15) | (river_shift > 0.15), 1.0, 0.0)

# Create a heatmap of the river shift
plt.figure(figsize=(8, 6))
plt.title("Heatmap of River Shift")
plt.imshow(river_shift, cmap='coolwarm', vmin=-1, vmax=1, alpha=alpha_mask)  
plt.colorbar(label="Shift Intensity")
plt.show()


In [ ]:
# Create a heatmap of the river shift in the right bounds therefore everything additional must be removed
plt.figure(figsize=(8, 6))
plt.imshow(river_shift, cmap='coolwarm', vmin=-1, vmax=1, alpha=alpha_mask)  
plt.axis("off")
plt.gca().set_position([0,0,1,1])
plt.savefig("river_shift_overlay.png", dpi=300, bbox_inches="tight", pad_inches=0, transparent=True)

plt.show()
plt.close()

In [ ]:
# Bounding box for the image
bounds = [[47.55891195678332, 11.437948183612978], [47.56320074444652, 11.484581611702358]]

m = folium.Map(location=[47.561, 11.461], zoom_start=14)

# Add the base layer (OpenStreetMap) and the overlay 
ImageOverlay(
    image="rivershift_overlay.png",  
    bounds=bounds,
    opacity=0.6,  
    interactive=True
).add_to(m)

# Add Layer Control in order to switch between layers
folium.LayerControl().add_to(m)

m.save(".html")
print("Map saved as '.html'")

In [ ]:
# Show map in VS Code
m